# `MEDUSA`
aka. Dynamic-Prospective LCA aka. Union(premise, temporalis)

In [1]:
from bw_temporalis import easy_timedelta_distribution, TemporalDistribution, easy_datetime_distribution
from bw_temporalis.lcia.climate import characterize_methane, characterize_co2
import sys
import os
sys.path.append(os.path.realpath('../'))
from medusa.edge_extractor import *
from medusa.matrix_modifier import MatrixModifier
from medusa.medusa_lca import *
from medusa.timeline_builder import TimelineBuilder
from medusa.dynamic_characterization import DynamicCharacterization
import bw2data as bd
import bw2calc as bc
import numpy as np
import pandas as pd

In [2]:
from tests.databases import *
db_abc_loopA_with_biosphere_advanced_simple()

100%|██████████| 2/2 [00:00<00:00, 3964.37it/s]


Vacuuming database 
Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 2/2 [00:00<?, ?it/s]


Vacuuming database 
Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 2/2 [00:00<?, ?it/s]


Vacuuming database 
Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 4/4 [00:00<?, ?it/s]


Vacuuming database 


In [3]:
demand = {('foreground', 'A'): 1}
gwp = ('GWP', 'example')

# Static LCA

In [4]:
slca = bc.LCA(demand, gwp)
slca.lci()
slca.lcia()
print(f'Static LCA score: {slca.score}')

Static LCA score: 240.5555559538397


# `MEDUSA` LCA

In [5]:
SKIPPABLE = [node.id for node in bd.Database('background_2020')] + [
    node.id for node in bd.Database('background_2024')
]

def filter_function(database_id: int) -> bool:
    return database_id in SKIPPABLE

Now we can do the graph traversal and create a timeline of edges:

In [6]:
import warnings
warnings.filterwarnings("ignore")

database_date_dict = {
            'background_2008': datetime.strptime("2008", "%Y"),
            'background_2024': datetime.strptime("2024", "%Y"),
            'foreground': "dynamic", # flag databases that should be temporally distributed with "dynamic"
        }

In [7]:
mlca = MedusaLCA(demand, gwp, filter_function, database_date_dict, max_calc=1000)

Starting graph traversal
Calculation count: 42


In [8]:
mlca.build_timeline()

All databases in database_date_dict exist as brightway project databases


,hash_producer,time_mapped_producer,date_producer,producer,producer_name,hash_consumer,time_mapped_consumer,date_consumer,consumer,consumer_name,amount,interpolation_weights
0,2024,258,2024-01-01,258,C,2024,266,2024-01-01,262,E,11.0,{'background_2024': 1}
1,2024,258,2024-01-01,258,C,2024,268,2024-01-01,264,B,13.0,{'background_2024': 1}
2,2024,266,2024-01-01,262,E,2024,267,2024-01-01,263,D,2.0,{'background_2024': 1}
3,2024,267,2024-01-01,263,D,2024,269,2024-01-01,265,A,0.5,{'background_2024': 1}
4,2024,268,2024-01-01,264,B,2024,266,2024-01-01,262,E,8.0,{'background_2024': 1}
5,2024,268,2024-01-01,264,B,2024,267,2024-01-01,263,D,5.0,{'background_2024': 1}
6,2024,268,2024-01-01,264,B,2024,269,2024-01-01,265,A,4.0,{'background_2024': 1}
7,2024,269,2024-01-01,265,A,2024,-1,2024-01-01,-1,-1,1.0,{'background_2024': 1}
8,2024,269,2024-01-01,265,A,2024,266,2024-01-01,262,E,0.1,{'background_2024': 1}


In [9]:
mlca.build_datapackage()

In [10]:
mlca.lci()

In [11]:
mlca.lcia()

In [12]:
print('New MEDUSA LCA Score:', mlca.score)
print('Old static LCA Score:', slca.score)

New MEDUSA LCA Score: 240.5555559538397
Old static LCA Score: 240.5555559538397


In [13]:
mlca.build_dynamic_biosphere()

In [14]:
mlca.calculate_dynamic_lci()

In [16]:
mlca.dynamic_inventory

{'CO2': {'time': array(['2005-12-31T12:21:36', '2008-01-01T00:00:00',
         '2008-12-31T05:49:12', '2009-12-31T11:38:24',
         '2021-12-31T12:21:36', '2024-01-01T00:00:00',
         '2024-12-31T05:49:12', '2025-12-31T11:38:24'],
        dtype='datetime64[s]'),
  'amount': array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         18.88888892,  0.        ,  0.        ])},
 'CH4': {'time': array(['2021-12-31T12:21:36'], dtype='datetime64[s]'),
  'amount': array([16.11111114])}}

In [17]:
mlca.characterize_dynamic_lci({"carbon dioxide": characterize_co2, "methane": characterize_methane})	 


In [18]:
mlca.characterized_inventory

,date,amount,flow,activity,amount_sum
0,2005-12-31 12:21:36,0.000000e+00,256,76867,0.000000e+00
1,2006-12-31 18:10:48,0.000000e+00,256,76867,0.000000e+00
2,2008-01-01 00:00:00,0.000000e+00,256,76867,0.000000e+00
3,2008-01-01 00:00:00,0.000000e+00,256,76867,0.000000e+00
4,2008-12-31 05:49:12,0.000000e+00,256,76867,0.000000e+00
...,...,...,...,...,...
95,2116-12-31 13:15:36,1.654957e-15,257,76867,4.184373e-11
96,2117-12-31 19:04:48,1.526732e-15,257,76867,4.184526e-11
97,2119-01-01 00:54:00,1.408442e-15,257,76867,4.184666e-11
98,2120-01-01 06:43:12,1.299318e-15,257,76867,4.184796e-11


In [20]:
mlca = MedusaLCA(demand, gwp, filter_function, database_date_dict, max_calc=1000)
mlca.build_timeline()
mlca.build_datapackage()
mlca.lci()
mlca.lcia()

mlca.build_dynamic_biosphere()
mlca.calculate_dynamic_lci()
mlca.characterize_dynamic_lci({"carbon dioxide": characterize_co2, "methane": characterize_methane})

Starting graph traversal
Calculation count: 42
All databases in database_date_dict exist as brightway project databases


TODO make sure all arguments can be passed, e.g. cumsum. switch between methane and CO2,
 addition of methane and CO2, selected timeframe, add units, add plotting, add ids sum needs to be the same as for temporalis example